# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 3-1. Задача TSP: инкрементальные алгоритмы.

В этой задаче Вам предлагается сравнить алгоритмы Nearest Neighbour и Nearest Insertion в задаче Euclidean TSP.

**Даны:**
* Координаты точек плоскости, являющихся вершинами графа.

**Найти:**
* Перестановку вершин, задающих минимальный по длине гамильтонов цикл в графе.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP.
* Реализуйте функции `solve_tsp_nearest_neighbour` и `solve_tsp_nearest_insertion`.
* Запустите функцию `run_all()`, чтобы протестировать свой код и сравнить качество решений, получаемых Nearest Neighbour и Nearest Insertion. Сильно ли они отличаются? Запишите свои качественные выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [11]:

import numpy as np
def read_tsp_instance(filename):
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0,0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates

def euclidean_distance(point1: tuple, point2: tuple) -> float:
    distance = (np.sum((np.array(point1) - np.array(point2))**2))**(1/2)
    return distance
def euclidean_distance_NP_sq(p: np.array, q: np.array) -> float:
    distance = np.sqrt((p[0] - q[0]) ** 2 + (p[1] - q[1]) ** 2)
    return distance

def calculate_tour_length(instance, permutation):
    n = len(permutation)
    return sum(euclidean_distance(instance[permutation[i]], 
                                  instance[permutation[(i+1) % n]]) for i in range(len(permutation)))

In [12]:
def solve_tsp_nearest_neighbour(instance):
    waitForVisit = np.array(range(1, len(instance)))
    ans = np.array([0]) 
    for it in range(1, len(instance)):
        distances = np.array([euclidean_distance(ans[-1], waitForVisit[kt]) for kt in range(len(instance) - it)])
        return np.insert(ans, it, waitForVisit[np.argmin(distances)])

In [13]:
def solve_tsp_nearest_insertion(instance):
    ans = np.array([0])
    tmp = np.array([list(k) for k in instance])
    waitForVisit = np.array(range(1, len(tmp)))
    for l in range(1, len(tmp)):
        In = tmp[ans]
        arr1 = np.array([x.T for x in (np.tile(In, len(tmp) - l).reshape((2, len(tmp) - l, l)))])
        Out = tmp[waitForVisit]
        arr2 = np.tile(Out, l).reshape((2, l, len(tmp) - l))
        D = euclidean_distance_NP_sq(arr1, arr2)
        InEd = (np.argmin((D + np.roll(D, 1, axis=0)).T - 
                                  np.array([euclidean_distance(tmp[ans[k - 1]], ans[k]) 
                                            for k in range(l)]))) % l
        OutEd = (np.argmin((D + np.roll(D, 1, axis=0)).T - 
                                    np.array([euclidean_distance(tmp[ans[k - 1]], ans[k]) 
                                              for k in range(l)]))) // l
        return np.insert(ans, InEd, waitForVisit[OutEd])

In [14]:
import time
from os.path import exists

def run_all():
    instance_filenames = ['d198.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp', 'pr107.tsp', 'pr152.tsp', 'pr439.tsp']
    for filename in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality_nn = calculate_tour_length(instance, solve_tsp_nearest_neighbour(instance))
        time_nn = time.monotonic()-time_start
        time_start = time.monotonic()
        quality_ni = calculate_tour_length(instance, solve_tsp_nearest_insertion(instance))
        time_ni = time.monotonic()-time_start
        print(' done in {:.2} seconds with tour length {} using NN and in {:.2} seconds with tour length {} using NI'.format(time_nn, int(quality_nn), time_ni, int(quality_ni)))

In [15]:
run_all()

Solving instance d198.tsp… done in 0.0029 seconds with tour length 2277 using NN and in 0.00064 seconds with tour length 2277 using NI
Solving instance d493.tsp… done in 0.018 seconds with tour length 3828 using NN and in 0.0019 seconds with tour length 5707 using NI
Solving instance d657.tsp… done in 0.021 seconds with tour length 2633 using NN and in 0.0019 seconds with tour length 3647 using NI
Solving instance d2103.tsp… done in 0.044 seconds with tour length 2201 using NN and in 0.0017 seconds with tour length 3714 using NI
Solving instance pr107.tsp… done in 0.0011 seconds with tour length 800 using NN and in 0.0003 seconds with tour length 4600 using NI
Solving instance pr152.tsp… done in 0.0015 seconds with tour length 2300 using NN and in 0.00039 seconds with tour length 2300 using NI
Solving instance pr439.tsp… done in 0.004 seconds with tour length 538 using NN and in 0.00057 seconds with tour length 1856 using NI


## Выводы
Если сравнивать алгоритм ближайших вставок, то видим, что для большей части тестов он работает хуже на конкретно этих тестах. Его энергопотребление в полтора раза больше, по сравнению с алгоритмом ближайшего соседа.
В то же время алгоритм ближайшего соседа в разы проще реализуем, чем алгоритм ближайших вставок, который применим больше в случае, когда мы не знаем принципиальных различий. На практике же чаще используется алгоритм ближайшего соседа с различными эвристиками. 